Nome: Fabio Grassiotto  
RA: 890441

# Aula 9_10 - Implementar a ReAct  com LLaMa 3 70B (groq)

- Testar no dataset do IIRC - 50 primeiras perguntas com resposta (test_questions.json em anexo)  
- Usar o prompt do LLaMAIndex: https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py  
- Salvar as respostas finais das 50 perguntas no JSON para exercício futuro de avaliação  
- Instruir o modelo a seguir a sequência Thougth, Action, Input, Observation (a observação não é do próprio modelo, mas resultado da busca)  
- É necessário usar o parâmetro stop_sequence="Observation:", para o modelo parar de gerar texto e esperar o retorno da busca. Implementem o código da busca e retornem os top-k documentos pro modelo (sugestão: k=5).  
- Instruir o modelo agir passo-a-passo (decomposição da pergunta).  
- Podem usar o LangChain, LLaMAindex ou outro framework. Ou implementar na mão.  
- Usar a busca como ferramenta  
- Usar o BM25 como buscador (repetir indexação do exercício passado)  
- Usar a indexação do Visconde: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipyn  


## Setup Environment
### Install packages

In [1]:
%%capture
%pip install -q torch
%pip install groq
%pip install llama-index-llms-groq
%pip install langchain
%pip install langchain-groq
%pip install -U sentence-transformers
%pip install faiss-cpu
%pip install rank_bm25
%pip install bs4
%pip install spacy
!python -m spacy download en_core_web_sm

### Imports

In [2]:
import os
import sys
import json
import torch
from collections import Counter
import string
import re
from bs4 import BeautifulSoup
from rank_bm25 import BM25Okapi
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm
import spacy
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

import warnings
warnings.simplefilter('ignore')

/home/fabiogr/miniconda3/envs/ml_rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Collab setup

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_9_10"
    os.chdir(project_folder)
    !ls -la

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Globals

In [4]:
NUM_QUESTIONS = 50
LLM_MODEL_NAME="llama3-70b-8192"
SEARCH_MODEL_NAME = "sentence-transformers/msmarco-MiniLM-L-6-v3"

## IIRC Dataset

In [5]:
if not os.path.exists("dataset/context_articles.json"):
    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
    !tar -xzf context_articles.tar.gz --directory dataset
    !rm context_articles.tar.gz

In [6]:
test_dataset  = json.load(open('dataset/test_questions.json', 'r'))
articles = json.load(open('dataset/context_articles.json', 'r'))

In [7]:
test_dataset[0]

{'answer': {'type': 'span',
  'answer_spans': [{'text': 'sky and thunder god',
    'passage': 'zeus',
    'type': 'answer',
    'start': 83,
    'end': 102}]},
 'question': 'What is Zeus know for in Greek mythology?',
 'context': [{'text': 'he Palici the sons of Zeus',
   'passage': 'main',
   'indices': [684, 710]},
  {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
  {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
   'passage': 'Zeus',
   'indices': [0, 110]}],
 'question_links': ['Greek mythology', 'Zeus'],
 'title': 'Palici'}

### Load and parse

In [8]:
# Grab the first 50 questions  # type: ignorewith an answer
def grab_documents():

  questions_found = []
  num_questions_found = 0
  documents = []
  all_titles = []

  for item in test_dataset:
    question = item['question']
    answer = item['answer']
    answer_type = answer['type']

    if answer_type == 'binary' or answer_type == 'value':
      final_answer = answer['answer_value']
    elif answer_type == 'span':
      final_answer = answer['answer_spans'][0]['text']
    elif answer_type == 'none':
      final_answer = 'none'
    else:
      final_answer = 'An error perhaps, bad type'
      print(answer_type)

    if (final_answer == 'none'):
      # Skip this one.
      continue
    else:
      # Thats a good question.
      
      # First add some extra info in the context-part
      for context in item['context']:
        if context['passage'] == "main":
            # Cleanup html tags
            soup = BeautifulSoup(context['text'], 'html.parser')
            clean_text = soup.get_text()

            documents.append({
                    "title": item['title'].lower(),
                    "content": clean_text
              }
          )
        all_titles.append(item['title'].lower())

      # And then grab the text from the articles json 
      for link in item["question_links"]:
        if link.lower() in articles and link.lower() not in all_titles:
          # Cleanup html tags
          soup = BeautifulSoup(articles[link.lower()], 'html.parser')
          clean_text = soup.get_text()

          documents.append({
            "title": link,
            "content": clean_text
          })
        all_titles.append(link.lower())

      questions_found.append({"Question": question, "Answer": final_answer})
      num_questions_found += 1
      if (num_questions_found == NUM_QUESTIONS):
        # found our questions
        break

  return questions_found, documents, all_titles

questions_to_ask, documents, all_titles = grab_documents()

### Sequence segmentation (3 per document)

In [9]:
contents_list = []
nlp = spacy.load('en_core_web_sm')

for t in tqdm(documents):
    paragraph = t.get('content')
    paragraph = paragraph[:512] 

    sentences = list(sent.text for sent in nlp(paragraph).sents)
    sentences = sentences[:3]
    excerpt = ""
    for s in sentences:
        excerpt += s
    contents_list.append(excerpt)

100%|██████████| 124/124 [00:01<00:00, 114.66it/s]


### Inspecting data

In [10]:
len(questions_to_ask)

50

In [11]:
len(contents_list)

124

In [12]:
contents_list[:10]

['he Palici the sons of Zeus',
 'in Greek mythology',
 "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greece and its civilization, and to gain understanding of the",
 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n',
 'he became aide-de-camp to King Victor Emm

In [13]:
all_titles[:10]

['palici',
 'palici',
 'palici',
 'greek mythology',
 'zeus',
 'giovanni messe',
 'giovanni messe',
 'world war i',
 'giovanni messe',
 'giovanni messe']

In [14]:
questions_to_ask[0]

{'Question': 'What is Zeus know for in Greek mythology?',
 'Answer': 'sky and thunder god'}

## FAISS Indexing

In [15]:
model = SentenceTransformer(SEARCH_MODEL_NAME)
model.to(device)

embeddings = model.encode(contents_list, show_progress_bar=True)

Batches: 100%|██████████| 4/4 [00:00<00:00, 10.01it/s]


In [16]:
# Create a FAISS index from the embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## Langchain Groq API

In [17]:
def load_groq_key():
    try:
        # Open and read the entire content of the file
        with open("groq-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None
    
chat = ChatGroq(temperature=0, groq_api_key=load_groq_key(), model_name=LLM_MODEL_NAME)

In [18]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Interactive Systems**: In applications like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, creating a more natural and engaging user experience. This is particularly important in customer-facing applications where delayed responses can lead to frustration and abandonment.\n2. **Real-time Decision Making**: In areas like finance, healthcare, and cybersecurity, timely decision-making is critical. Low-latency LLMs can quickly analyze vast amounts of data, enabling swift identification of patterns, anomalies, and insights, which inform timely decisions.\n3. **Live Content Generation**: Low-latency LLMs are essential for generating content in real-time, such as live subtitles, closed captions, or automated summarization of

## ReAct Agent Implementation

### Prompt from Langchain (https://smith.langchain.com/hub/hwchase17/react-chat)

This prompt was modified to remove the chat history, so every question is a new chat with the LLM.

In [19]:
LANGCHAIN_CHAT_SYSTEM_HEADER = """\

Assistant is a large language model trained by Meta AI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
Assistant has access to the following tools:
{tools}
To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
Begin!
New input: {input}
{agent_scratchpad}
"""

### Agent tool definition

Top 5 and top 10 hits returned from the search.

In [20]:
@tool
def get_top5(question: str) -> list[str]:
    """Returns a list of five strings with texts to help solve a question."""
    xq = model.encode([question])
    _, I = index.search(xq, 5)  # search

    results = []
    for i in range(0, 5):
        results.append(contents_list[I[0][i]])    
    return results

@tool
def get_top10(question: str) -> list[str]:
    """Returns a list of ten strings with texts to help solve a question."""
    xq = model.encode([question])
    _, I = index.search(xq, 10)  # search

    results = []
    for i in range(0, 10):
        results.append(contents_list[I[0][i]])    
    return results

In [21]:
print(get_top5.name)
print(get_top5.description)
print(get_top5.args)

get_top5
get_top5(question: str) -> list[str] - Returns a list of five strings with texts to help solve a question.
{'question': {'title': 'Question', 'type': 'string'}}


In [ ]:
print(get_top10.name)
print(get_top10.description)
print(get_top10.args)

## Execution Run

### Evaluation functions

In [23]:
def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

### Analysing the output for 50 questions

In [24]:
def evaluate(executor):
    # Create dataframe with question, answer, Llm answer and scores
    df_results = pd.DataFrame(columns=['question', 'answer', 'LLM answer', 'F1', 'Exact Match'])

    for item in tqdm(questions_to_ask):
        question = item.get('Question')
        answer = normalize_answer(item.get('Answer'))

        # RAG context using search
        agent_answer = executor.invoke({"input": "Question: " + question})
        llm_answer = normalize_answer(agent_answer['output'])
        f1_score = token_f1_score(llm_answer, answer)
        e_match_score = 1 if llm_answer == answer else 0
        
        row = pd.Series([question, answer, llm_answer, f1_score, e_match_score], index=df_results.columns)
        df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

    return df_results

### Searching with Tool returning top 5 hits

In [25]:
prompt = PromptTemplate.from_template(LANGCHAIN_CHAT_SYSTEM_HEADER)
llm = ChatGroq(temperature=0,
               model_name=LLM_MODEL_NAME,
               api_key=load_groq_key())
tools = [get_top5]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
df_top5 = evaluate(agent_executor)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', 'in Greek mythology', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions o

  2%|▏         | 1/50 [00:02<01:51,  2.28s/it]

Thought: Do I need to use a tool? No
Final Answer: Zeus is known as the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'The 1936 Summer Olympics (German: Olympische Sommerspiele 1936), officially known as the Games of the XI Olympiad, was an international multi-sport event held in 1936 in Berlin, Nazi Germany.Berlin won the bid to host the Games over Barcelona, Spain, on 26 April 1931, at the 29th IOC Session in Barcelona.It marked the second and final time the International Olympic Committee gathered to vote in a city that was bidding to host those Games.\n\n', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the

  4%|▍         | 2/50 [00:06<02:37,  3.29s/it]

...

Do I need to use a tool? No
Final Answer: Based on the provided information, Eduard van Beinum passed away on April 13, 1959. Gilmore's retirement date is not specified, but it is likely that Eduard van Beinum was still alive when Gilmore retired, assuming Gilmore retired before 1959.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right win

 98%|█████████▊| 49/50 [09:31<00:13, 13.81s/it]

Do I need to use a tool? No
Final Answer: The Battle of Fort Donelson had a higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'In the Battle 

100%|██████████| 50/50 [09:43<00:00, 11.67s/it]

Do I need to use a tool? No
Final Answer: The country that some of the tribes escaped to after their deportation from northern Mexico is the United States, specifically southern Arizona.

> Finished chain.


#### Results - Top 5 

In [26]:
df_top5

,question,answer,LLM answer,F1,Exact Match
0,What is Zeus know for in Greek mythology?,sky and thunder god,zeus is known as sky and thunder god in ancien...,0.320000,0
1,How long had the First World War been over whe...,5,based on provided information it seems that me...,0.057143,0
2,How old was Messe when the First World War sta...,30,unfortunately provided information does not me...,0.000000,0
3,How long had Angela Scoular been acting profes...,2,based on information provided angela scoular w...,0.000000,0
4,What is the capacity of the stadium where Brun...,26688,capacity of stadium where brunt returned to ac...,0.000000,0
5,Which stadium where Brunt played can hold more...,white hart lane,white hart lane can hold more people with capa...,0.272727,0
6,In which country was Wilhelm Müller born?,germany,wilhelm müller was born in germany,0.285714,0
7,Which battle Wilhelm Müller fought in while in...,kulm,battle wilhelm müller fought in while in pruss...,0.111111,0
8,How much time had passed between Wilhelm's ret...,9,based on information provided it appears that ...,0.045455,0
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm max müller 15 may 1862 – july 1919 liv...,0.210526,0


In [27]:
df_top5.to_csv("results/react_results_top5.csv")

In [28]:
print("Metrics:")
print(f"F1 score: Avg: {df_top5['F1'].mean():.2f}. Std: {df_top5['F1'].std():.2f}")
print(f"Exact Match score: Avg: {df_top5['Exact Match'].mean():.2f}. Std: {df_top5['Exact Match'].std():.2f}")

Metrics:
F1 score: Avg: 0.10. Std: 0.12
Exact Match score: Avg: 0.00. Std: 0.00


### Searching with Tool returning top 10 hits

In [29]:
prompt = PromptTemplate.from_template(LANGCHAIN_CHAT_SYSTEM_HEADER)
llm = ChatGroq(temperature=0,
               model_name=LLM_MODEL_NAME,
               api_key=load_groq_key())
tools = [get_top10]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
df_top10 = evaluate(agent_executor)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', 'in Greek mythology', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions 

  2%|▏         | 1/50 [00:17<14:00, 17.16s/it]

Do I need to use a tool? No
Final Answer: Zeus is known as the king of the gods in Greek mythology, and is often associated with the sky and thunder.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'The 1936 Summer Olympics (German: Olympische Sommerspiele 1936), officially known as the Games of the XI Olympiad, was an international multi-sport event held in 1936 in Berlin, Nazi Germany.Berlin won the bid to host the Games over Barcelona, Spain, on 26 April 1931, at the 29th IOC Session in Barcelona.It marked the second and final time the International Olympic Committee gathered to vote in a city that was bidding to host those Games.\n\n', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the 

  4%|▍         | 2/50 [00:35<14:02, 17.56s/it]

...

Thought: Do I need to use a tool? No
Final Answer: Based on the provided information, Eduard van Beinum passed away on April 13, 1959. Gilmore is not mentioned in the provided text, but I assume you are referring to Artis Gilmore, the American basketball player. If that's correct, Artis Gilmore was born on September 21, 1949, which means Eduard van Beinum was already deceased when Gilmore retired.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prin

 98%|█████████▊| 49/50 [12:50<00:18, 18.06s/it]

Do I need to use a tool? No
Final Answer: The Battle of Fort Donelson had a higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'In the Battle

100%|██████████| 50/50 [13:07<00:00, 15.75s/it]

Do I need to use a tool? No
Final Answer: The country that some of the tribes escaped to after their deportation from northern Mexico is the United States, specifically southern Arizona.

> Finished chain.


#### Results - Top 10

In [30]:
df_top10

,question,answer,LLM answer,F1,Exact Match
0,What is Zeus know for in Greek mythology?,sky and thunder god,zeus is known as king of gods in greek mytholo...,0.272727,0
1,How long had the First World War been over whe...,5,based on provided information it appears that ...,0.057143,0
2,How old was Messe when the First World War sta...,30,question is asking about messe but information...,0.000000,0
3,How long had Angela Scoular been acting profes...,2,angela scoular had been acting professionally ...,0.000000,0
4,What is the capacity of the stadium where Brun...,26688,capacity of stadium where brunt returned to ac...,0.100000,0
5,Which stadium where Brunt played can hold more...,white hart lane,white hart lane can hold more people with capa...,0.272727,0
6,In which country was Wilhelm Müller born?,germany,wilhelm müller was born in dessau germany,0.250000,0
7,Which battle Wilhelm Müller fought in while in...,kulm,battle wilhelm müller fought in while in pruss...,0.111111,0
8,How much time had passed between Wilhelm's ret...,9,based on provided information wilhelm returned...,0.048780,0
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm max müller 15 may 1862 – july 1919 was...,0.160000,0


In [31]:
df_top10.to_csv("results/react_results_top10.csv")

In [32]:
print("Metrics:")
print(f"F1 score: Avg: {df_top10['F1'].mean():.2f}. Std: {df_top10['F1'].std():.2f}")
print(f"Exact Match score: Avg: {df_top10['Exact Match'].mean():.2f}. Std: {df_top10['Exact Match'].std():.2f}")

Metrics:
F1 score: Avg: 0.16. Std: 0.24
Exact Match score: Avg: 0.06. Std: 0.24
